## setup

In [1]:
import torch

torch.cuda.empty_cache()
print(f"GPU: {torch.cuda.get_device_name(0)}")

from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
import transformers

from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM


GPU: NVIDIA GeForce RTX 4090


2024-01-07 17:35:06.468884: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-07 17:35:06.487526: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-07 17:35:06.487547: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-07 17:35:06.487560: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-07 17:35:06.491952: I tensorflow/core/platform/cpu_feature_g

In [2]:
!nvidia-smi
torch.cuda.empty_cache()
print(f"GPU: {torch.cuda.get_device_name(0)}")

Sun Jan  7 17:35:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:01:00.0  On |                  Off |
|  0%   50C    P8              36W / 450W |      6MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Preload parameters

In [3]:
base_model = "HuggingFaceH4/zephyr-7b-beta"
new_model = "zephyr-7b-padding"
output_dir = "./logs" # tensorboard結果

### bitsandbytes parameters for QLoRA

In [4]:
# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "bfloat16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

### LoRA parameters

In [5]:
# LoRA attention dimension (Rank)
lora_r = 64

# Alpha parameter for LoRA scaling (The weight matrix is scaled by 𝑙𝑜𝑟𝑎_𝑎𝑙𝑝ℎ𝑎 / 𝑙𝑜𝑟𝑎_𝑟𝑎𝑛𝑘)
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

### TrainingArguments parameters

In [6]:
# Output directory where the model predictions and checkpoints will be stored
output_dir = "./logs"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (llama2 use bf16)
fp16 = False
bf16 = True

# Batch size per GPU for training
per_device_train_batch_size = 2

# Batch size per GPU for evaluation
per_device_eval_batch_size = 2

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.2

# Initial learning rate (AdamW optimizer)
learning_rate = 5e-5

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

### SFT parameters

In [7]:
# Maximum sequence length to use
max_seq_length = 2048

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0} # or "auto"

## Load model

In [8]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

(True, True)

## load datasets

In [9]:
dataset = load_dataset("gsm8k",'main')

In [10]:
print(dataset['train'][0])
print(len(dataset['train']))

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}
7473


### Preprocessing data

In [11]:
def convert_to_target_format(data):
    return {'formatted_text': f"<|user|>{data['question']}\n<|assistant|>\n{data['answer']}</s>"}

In [12]:
train_dataset = dataset['train']
formatted_train_dataset = train_dataset.map(convert_to_target_format,remove_columns=["question","answer"])

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

In [13]:
print(formatted_train_dataset[0])

{'formatted_text': '<|user|>Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?\n<|assistant|>\nNatalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72</s>'}


### Testing Inference

In [19]:
import random
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
data_list = list(dataset['test'])  # 假设您想要从训练集中抽取数据
# 从列表中随机选择一条数据
random_data_point = random.choice(data_list)
print(f"question: {random_data_point['question']}\n")
print(f"answer: {random_data_point['answer']}\n")
# 现在 random_data_point 包含了您随机选择的一条数据
# print(random_data_point['question'])
print("----model_inferencing----")

prompt = random_data_point['question']
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<|user|>{prompt}</s>\n")
print(result[0]['generated_text'])

question: Ali had $21. Leila gave him half of her $100. How much does Ali have now?

answer: Leila gave 100/2 = $<<100/2=50>>50 to Ali.
Ali now has $21+ $50 = $<<21+50=71>>71.
#### 71

----model_inferencing----
<|user|>Ali had $21. Leila gave him half of her $100. How much does Ali have now?</s>
<|assistant|>
After Leila gives Ali half of her $100, Ali now has half of his original $21, plus the $50 he received from Leila.

Let's call Ali's original $21 as x.

Leila gives Ali half of her $100, which is $50.

Ali's new amount = x + 50

Now, we know that Leila's new amount is $50 less than her original $100.

Leila's new amount = $50

Leila's original amount - Leila's new amount = $100 - $50

100 - Leila's new amount = x


## Training Arguments

In [20]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    resume_from_checkpoint=True,
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_train_dataset,
    peft_config=peft_config,
    dataset_text_field="formatted_text", # 自己處理後的欄位
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)



Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

In [21]:
trainer.train()

{'loss': 1.5092, 'learning_rate': 1.1061946902654869e-05, 'epoch': 0.01}
{'loss': 1.8316, 'learning_rate': 2.2123893805309738e-05, 'epoch': 0.01}
{'loss': 1.1951, 'learning_rate': 3.3185840707964604e-05, 'epoch': 0.02}
{'loss': 1.1891, 'learning_rate': 4.4247787610619477e-05, 'epoch': 0.03}
{'loss': 0.8613, 'learning_rate': 4.999864732969518e-05, 'epoch': 0.03}
{'loss': 0.8681, 'learning_rate': 4.99871412250638e-05, 'epoch': 0.04}
{'loss': 0.8493, 'learning_rate': 4.9963899583683975e-05, 'epoch': 0.05}
{'loss': 0.8803, 'learning_rate': 4.9928933321295676e-05, 'epoch': 0.05}
{'loss': 0.8425, 'learning_rate': 4.9882258860260065e-05, 'epoch': 0.06}
{'loss': 0.7904, 'learning_rate': 4.982389812184646e-05, 'epoch': 0.07}
{'loss': 0.8548, 'learning_rate': 4.975387851593677e-05, 'epoch': 0.07}
{'loss': 0.8691, 'learning_rate': 4.9672232928152096e-05, 'epoch': 0.08}
{'loss': 0.804, 'learning_rate': 4.957899970440752e-05, 'epoch': 0.09}
{'loss': 0.8556, 'learning_rate': 4.9474222632902484e-05, 

TrainOutput(global_step=3737, training_loss=0.797121823519859, metrics={'train_runtime': 902.0186, 'train_samples_per_second': 8.285, 'train_steps_per_second': 4.143, 'train_loss': 0.797121823519859, 'epoch': 1.0})

## Save model

In [24]:
trainer.model.save_pretrained(new_model)

In [25]:
# Empty VRAM
del model
# del pipe
del trainer
import gc
gc.collect()
gc.collect()

0

## Reload model in FP16 and merge it with LoRA weights

In [8]:
# Reload model in FP16 and merge it with LoRA weights
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

## Save full model

In [9]:
model.save_pretrained(new_model+"-full") # 儲存完整模型
tokenizer.save_pretrained(new_model+"-full")

('zephyr-7b-padding-full/tokenizer_config.json',
 'zephyr-7b-padding-full/special_tokens_map.json',
 'zephyr-7b-padding-full/tokenizer.model',
 'zephyr-7b-padding-full/added_tokens.json',
 'zephyr-7b-padding-full/tokenizer.json')

## Inference

In [4]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
inf_model_path="./zephyr-7b-padding-full"
inf_model=AutoModelForCausalLM.from_pretrained(inf_model_path, device_map="auto")
inf_tokenizer = AutoTokenizer.from_pretrained(inf_model_path)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [38]:
# Run text generation pipeline with our next model
import random
# 将数据集转换为 Python 列表
data_list = list(dataset['test'])  # 假设您想要从训练集中抽取数据

# 从列表中随机选择一条数据
random_data_point = random.choice(data_list)

# 现在 random_data_point 包含了您随机选择的一条数据
# print(random_data_point['question'])


prompt = random_data_point['question']
pipe = pipeline(task="text-generation", model=inf_model, tokenizer=inf_tokenizer, max_length=200)
result = pipe(f"<|user|>{prompt}")
print(result[0]['generated_text'])

James decides to buy birthday candles for his 2 sons.  One of them is 12 and the other is 4 years younger.  A pack of 5 candles costs $3.  How much does James spend on candles?
<|assistant|>
The older son is 12 so the younger son is 12-4=<<12-4=8>>8 years old.
The older son needs 12 candles and the younger son needs 8 candles so they need 12+8=<<12+8=20>>20 candles.
20 candles cost $3 each so they cost 20*3=$<<20*3=60>>60.
#### 10. A store sells 1000 pairs of shoes.  The store sells 200 pairs of shoes on Monday, 300
